In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install ipycanvas
!pip install ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.8/125.8 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 20.6 MB/s eta 0:00:00


In [3]:
import tensorflow as tf
import numpy as np
import os
from tensorflow import keras
from tensorflow.keras import layers, models
from google.colab import output
from ipycanvas import Canvas
from PIL import Image
from IPython.display import display, clear_output
import ipywidgets as widgets

In [4]:
# Move to the dataset directory
%cd /content/drive/My Drive/dataset

# Set the base path and verify that it's pointing to the correct directory
base_path = os.getcwd()  # Corrected to os.getcwd() to retrieve the current directory path
train_dir = os.path.join(base_path, 'train')
val_dir = os.path.join(base_path, 'val')
test_dir = os.path.join(base_path, 'test')

print("Base Path:", base_path)
print("Train Directory:", train_dir)
print("Validation Directory:", val_dir)
print("Test Directory:", test_dir)



/content/drive/My Drive/dataset
Base Path: /content/drive/My Drive/dataset
Train Directory: /content/drive/My Drive/dataset/train
Validation Directory: /content/drive/My Drive/dataset/val
Test Directory: /content/drive/My Drive/dataset/test


In [ ]:
# ndak perlu di load
# Memuat data
def load_images_from_directory(directory):
    images = []
    labels = []

    # Iterasi melalui setiap subdirektori dan file di dalamnya
    for label in os.listdir(directory):
        label_dir = os.path.join(directory, label)

        # Pastikan kita hanya memuat direktori yang benar
        if os.path.isdir(label_dir):
            for filename in os.listdir(label_dir):
                if filename.endswith('.jpg'):
                    # Memuat gambar
                    img_path = os.path.join(label_dir, filename)
                    img = Image.open(img_path).convert('L')  # Mengubah gambar menjadi grayscale
                    img = img.resize((28, 28))  # Mengubah ukuran gambar jika diperlukan
                    img_array = np.array(img) / 255.0  # Normalisasi gambar
                    images.append(img_array)
                    labels.append(label)  # Menggunakan nama folder sebagai label

    return np.array(images), np.array(labels)

# Memuat data dari direktori
train_images, train_labels = load_images_from_directory(train_dir)
val_images, val_labels = load_images_from_directory(val_dir)
test_images, test_labels = load_images_from_directory(test_dir)

# Verifikasi bentuk data
print("Train Images Shape:", train_images.shape)
print("Train Labels Shape:", train_labels.shape)
print("Validation Images Shape:", val_images.shape)
print("Validation Labels Shape:", val_labels.shape)
print("Test Images Shape:", test_images.shape)
print("Test Labels Shape:", test_labels.shape)

KeyboardInterrupt: 

In [5]:
# Membuat model dengan CNN
model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(150, 150, 3)),
    tf.keras.layers.Rescaling(1./255),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(256, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(512, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(50, activation='softmax')
    ])

model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ rescaling (Rescaling)                │ (None, 150, 150, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 148, 148, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 74, 74, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 72, 72, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 36, 36, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 34, 34, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 17, 17, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 15, 15, 512)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 7, 7, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │       6,422,784 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 50)                  │          12,850 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 7,986,610 (30.47 MB)

 Trainable params: 7,986,610 (30.47 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
train_dataset = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=(150, 150),
    batch_size=128,
    label_mode='categorical'
    )

validation_dataset = tf.keras.utils.image_dataset_from_directory(
    val_dir,
    image_size=(150, 150),
    batch_size=128,
    label_mode='categorical'
    )

# Optimize the datasets for training
SHUFFLE_BUFFER_SIZE = 1000
PREFETCH_BUFFER_SIZE = tf.data.AUTOTUNE

train_dataset_final = (train_dataset
                       .cache()
                       .shuffle(SHUFFLE_BUFFER_SIZE)
                       .prefetch(PREFETCH_BUFFER_SIZE)
                       )

validation_dataset_final = (validation_dataset
                            .cache()
                            .prefetch(PREFETCH_BUFFER_SIZE)
                            )

Found 8000 files belonging to 50 classes.
Found 2000 files belonging to 50 classes.


In [7]:
from tensorflow.keras.optimizers import Adam
optimizer = Adam(learning_rate=0.001)
model.compile(loss = 'categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
print(optimizer)

In [ ]:
history = model.fit(
    train_dataset_final,
    epochs=20,
    validation_data = validation_dataset_final,
    verbose = 1
)

Epoch 1/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 2531s 26s/step - accuracy: 0.0302 - loss: 3.7274 - val_accuracy: 0.0380 - val_loss: 3.4342
Epoch 2/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 1463s 23s/step - accuracy: 0.0525 - loss: 3.4518 - val_accuracy: 0.2655 - val_loss: 2.6296
Epoch 3/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 21s/step - accuracy: 0.3635 - loss: 2.3670 

In [ ]:
# third party
output.enable_custom_widget_manager()

NameError: name 'output' is not defined

In [ ]:
# Deklarasi canvas sebagai variabel global
global canvas
canvas = None

# Fungsi untuk membuat canvas dan menggambar
def create_canvas():
    global canvas
    canvas = Canvas(width=280, height=280, sync_image_data=True)
    canvas.fill_style = '#9E7F62'  # Background coklat lontar
    canvas.fill_rect(0, 0, 280, 280)  # Menyiapkan latar belakang coklat lontar
    canvas.stroke_style = 'black'  # Warna garis hitam
    canvas.line_width = 10  # Lebar garis
    drawing = False

    # Fungsi untuk menangani mouse
    def handle_mouse_down(x, y):
        nonlocal drawing
        drawing = True
        canvas.begin_path()
        canvas.move_to(x, y)

    def handle_mouse_move(x, y):
        if drawing:
            canvas.line_to(x, y)
            canvas.stroke()

    def handle_mouse_up(x, y):
        nonlocal drawing
        drawing = False

    canvas.on_mouse_down(handle_mouse_down)
    canvas.on_mouse_move(handle_mouse_move)
    canvas.on_mouse_up(handle_mouse_up)

    return canvas

# Proses gambar dari canvas untuk prediksi
def process_canvas_for_prediction(canvas):
    img_data = canvas.get_image_data(0, 0, canvas.width, canvas.height)
    img = np.array(img_data.data, dtype=np.uint8).reshape((canvas.height, canvas.width, 4))[:, :, 0]
    img = Image.fromarray(img)
    img = img.convert('L')
    img = img.resize((150, 150))
    img = np.array(img) / 255.0  # Normalisasi
    # Duplicate channel to create 3 channels (RGB)
    img = img.reshape(1, 150, 150, 1)
    return img

# Fungsi untuk melakukan prediksi
def classify():
    image = save_canvas()
    image = tf.keras.utils.img_to_array(image)
    image = np.expand_dims(image, axis=0)

    prediction = model.predict(image, verbose=0)[0]
    prediction_index = np.argmax(prediction)

    classes = ['YA','WA','ULU','TEDONG','TALENG','TA','SUKU','SA','RA','PA',
               'NYA','NGA','NA','MA','LA','KA','JA','GEMPELAN-SA-SAPA',
               'GEMPELAN-PA','GANTUNGAN-TA','GANTUNGAN-NYA','GANTUNGAN-NGA',
               'GANTUNGAN-NA','GANTUNGAN-MA','GANTUNGAN-LA','GANTUNGAN-KA',
               'GANTUNGAN-JA','GANTUNGAN-GA','GANTUNGAN-DA','GANTUNGAN-CA',
               'GANTUNGAN-BA','GANTUNGAN-A','GA','DA','CECEK','CA',
               'BISAH','BA','ADEG-ADEG','A','9','8','7','6','5','4','3','2','1','0']

    predicted_class = classes[prediction_index]

    clear_output(wait=True)
    display(canvas)
    print(f'Predicted class: {predicted_class}')

# Tombol untuk mengklasifikasikan
from ipywidgets import Button

button = Button(description="Classify")
button.on_click(lambda x: classify())
display(button)

# Tombol untuk menghapus gambar
clear_button = widgets.Button(description="Clear")
display(clear_button)

def on_clear_click(b):
    canvas.clear()  # Bersihkan canvas untuk menggambar ulang
    canvas.fill_style = '#9E7F62'
    canvas.fill_rect(0, 0, 280, 280)

clear_button.on_click(on_clear_click)

# Jalankan fungsi utama untuk menerima input tulisan tangan
run_classification(model)

NameError: name 'widgets' is not defined

In [ ]:
from google.colab import output
output.disable_custom_widget_manager()